In [85]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from joblib import dump
import holidays
from datetime import datetime

### Location Config

In [17]:
location = "lovrenska-jezera"
revision = 2

### Data Preprocessing

In [39]:
features = ["temperature", "rain", "month", "day_of_month", "day_of_week", "is_national_holiday", "is_school_holiday"]
target = "count"

In [18]:
hikers_data = pd.read_csv(f"../data/hikers/test/{location}.csv")
weather_data = pd.read_csv(f"../data/weather/aggregated/{location}.csv")

In [19]:
# Cut off hikers data at 2024
hikers_data_cut = hikers_data[hikers_data["datum"] < "2024-01-01"]

In [7]:
# List of national holidays
national_holidays = holidays.SI()

In [8]:
# List of school holidays and code to check if a date is holiday

school_holidays = [["2021-12-25", "2022-01-02"], ["2022-02-21", "2022-02-25"], ["2022-02-28", "2022-03-04"], ["2022-04-27", "2022-05-02"], ["2022-06-25", "2022-08-31"], ["2022-10-31", "2022-11-04"], ["2022-12-26", "2023-01-02"], ["2023-01-30", "2023-02-01"], ["2023-02-06", "2023-02-10"], ["2023-04-27", "2023-05-02"], ["2023-06-26", "2023-08-31"],["2023-10-30", "2023-11-3"], ["2023-12-25", "2024-01-02"], ["2024-02-19", "2024-02-23"], ["2024-02-26", "2023-03-01"], ["2024-4-27", "2023-05-02"], ["2024-06-26", "2024-08-31"]]
school_holidays = [[datetime.strptime(start, "%Y-%m-%d"), datetime.strptime(end, "%Y-%m-%d")] for start, end in school_holidays]

def is_school_holiday(date):
    for start, end in school_holidays:
        if start <= date <= end:
            return True
    return False

In [46]:
# Calculate our features and prepare data

def prepare_data(row):
    dt = datetime.fromisoformat(row["datetime"])
    row["month"] = dt.month
    row["day_of_month"] = dt.day
    row["day_of_week"] = dt.weekday()
    row["is_national_holiday"] = dt in national_holidays or dt.weekday() >= 5
    row["is_school_holiday"] = is_school_holiday(dt)

    if not (counts := hikers_data_cut.loc[hikers_data_cut["datum"] == row["datetime"]]).empty:
        row["count"] = counts["vhodi"].values[0] + counts["izhodi"].values[0]

    return row

data = weather_data.apply(prepare_data, axis=1).dropna()

In [47]:
# Data before filtering
data

,count,datetime,day_of_month,day_of_week,is_national_holiday,is_school_holiday,location,month,rain,temperature
177,318.0,2022-06-27,27,0,False,True,lovrenska-jezera,6,0.0,19.469444
178,66.0,2022-06-28,28,1,False,True,lovrenska-jezera,6,0.0,17.882639
179,214.0,2022-06-29,29,2,False,True,lovrenska-jezera,6,0.6,18.314583
180,288.0,2022-06-30,30,3,False,True,lovrenska-jezera,6,0.0,17.002778
181,379.0,2022-07-01,1,4,False,True,lovrenska-jezera,7,0.0,18.290278
...,...,...,...,...,...,...,...,...,...,...
725,8.0,2023-12-27,27,2,False,True,lovrenska-jezera,12,0.0,4.711806
726,2.0,2023-12-28,28,3,False,True,lovrenska-jezera,12,0.0,4.631250
727,9.0,2023-12-29,29,4,False,True,lovrenska-jezera,12,0.0,2.884722
728,15.0,2023-12-30,30,5,True,True,lovrenska-jezera,12,0.0,2.809028


In [44]:
# Filter out outliers

avg = data["count"].mean()
std_dev = data["count"].std()

is_not_outlier = data["count"] <= (avg + 2 * std_dev)
is_holiday = data["is_national_holiday"]

filtered_data = data[is_not_outlier | is_holiday]
removed_data = data[~(is_not_outlier | is_holiday)]

In [42]:
avg, std_dev

(289.48, 610.7518751252711)

In [48]:
# Remaining data after filtering
filtered_data

,count,datetime,day_of_month,day_of_week,is_national_holiday,is_school_holiday,location,month,rain,temperature
177,318.0,2022-06-27,27,0,False,True,lovrenska-jezera,6,0.0,19.469444
178,66.0,2022-06-28,28,1,False,True,lovrenska-jezera,6,0.0,17.882639
179,214.0,2022-06-29,29,2,False,True,lovrenska-jezera,6,0.6,18.314583
180,288.0,2022-06-30,30,3,False,True,lovrenska-jezera,6,0.0,17.002778
181,379.0,2022-07-01,1,4,False,True,lovrenska-jezera,7,0.0,18.290278
...,...,...,...,...,...,...,...,...,...,...
725,8.0,2023-12-27,27,2,False,True,lovrenska-jezera,12,0.0,4.711806
726,2.0,2023-12-28,28,3,False,True,lovrenska-jezera,12,0.0,4.631250
727,9.0,2023-12-29,29,4,False,True,lovrenska-jezera,12,0.0,2.884722
728,15.0,2023-12-30,30,5,True,True,lovrenska-jezera,12,0.0,2.809028


In [49]:
# Removed data after filtering
removed_data

,count,datetime,day_of_month,day_of_week,is_national_holiday,is_school_holiday,location,month,rain,temperature
325,1922.0,2022-11-22,22,1,False,False,lovrenska-jezera,11,7.7,-2.220139
341,1801.0,2022-12-08,8,3,False,False,lovrenska-jezera,12,0.8,-3.013889
342,1613.0,2022-12-09,9,4,False,False,lovrenska-jezera,12,28.5,1.044444
430,7431.0,2023-03-07,7,1,False,False,lovrenska-jezera,3,8.0,-1.680556
667,2259.0,2023-10-30,30,0,False,True,lovrenska-jezera,10,1.8,8.693750
670,2409.0,2023-11-02,2,3,False,True,lovrenska-jezera,11,8.4,5.093750
698,4598.0,2023-11-30,30,3,False,False,lovrenska-jezera,11,9.4,-0.369444
703,1549.0,2023-12-05,5,1,False,False,lovrenska-jezera,12,4.1,-2.345139


In [53]:
# Prepare x and y data
x_data = filtered_data[features]
y_data = filtered_data[target]

In [54]:
x_data

,temperature,rain,month,day_of_month,day_of_week,is_national_holiday,is_school_holiday
177,19.469444,0.0,6,27,0,False,True
178,17.882639,0.0,6,28,1,False,True
179,18.314583,0.6,6,29,2,False,True
180,17.002778,0.0,6,30,3,False,True
181,18.290278,0.0,7,1,4,False,True
...,...,...,...,...,...,...,...
725,4.711806,0.0,12,27,2,False,True
726,4.631250,0.0,12,28,3,False,True
727,2.884722,0.0,12,29,4,False,True
728,2.809028,0.0,12,30,5,True,True


In [55]:
y_data

177     318.0
178      66.0
179     214.0
180     288.0
181     379.0
        ...  
725       8.0
726       2.0
727       9.0
728      15.0
729    1268.0
Name: count, Length: 542, dtype: float64

In [60]:
# Split datasets into train and test
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

# TODO: Possible improvement: Maybe encode categorical features ("month", "day_of_month", "day_of_week", etc.) using one-hot encoding

# Normalize datasets with scaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [66]:
# Convert to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float64)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float64).view(-1, 1)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float64)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float64).view(-1, 1)

### Prepare Model

In [69]:
class HikerPredictor(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size, hidden_size_1)
        self.relu1 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size_2, output_size)
        self.softplus = nn.Softplus()

        self.double()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.softplus(x)
        return x

In [70]:
# Instantiate the model
input_size = len(features)
hidden_size_1 = 64
hidden_size_2 = 32
output_size = 1
model = HikerPredictor(input_size, hidden_size_1, hidden_size_2, output_size)

In [71]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

### Train Model

In [72]:
# Training loop
num_epochs = 100000
for epoch in range(num_epochs):
    optimizer.zero_grad()

    outputs = model(x_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{num_epochs}: {loss}")

    loss.backward()
    optimizer.step()

Epoch 0/1000000: 309901.55889090657
Epoch 100/1000000: 309778.0526018031
Epoch 200/1000000: 309518.1580255806
Epoch 300/1000000: 308973.82309567934
Epoch 400/1000000: 308023.43280780286
Epoch 500/1000000: 306775.6716751815
Epoch 600/1000000: 304811.98908008763
Epoch 700/1000000: 302553.85584429523
Epoch 800/1000000: 300109.4076189793
Epoch 900/1000000: 296909.94061488356
Epoch 1000/1000000: 292420.455387016
Epoch 1100/1000000: 288176.0885160654
Epoch 1200/1000000: 282683.38995115826
Epoch 1300/1000000: 277119.13338289916
Epoch 1400/1000000: 271253.4014976106
Epoch 1500/1000000: 264325.09314495174
Epoch 1600/1000000: 258018.58062180286
Epoch 1700/1000000: 249595.9066713348
Epoch 1800/1000000: 243158.10013002434
Epoch 1900/1000000: 235819.03468941216
Epoch 2000/1000000: 228681.11863001916
Epoch 2100/1000000: 222778.01388557986
Epoch 2200/1000000: 215310.63502064205
Epoch 2300/1000000: 206857.14041522317
Epoch 2400/1000000: 203023.00411502528
Epoch 2500/1000000: 196636.46261369117
Epoch 2

KeyboardInterrupt: 

### Evaluate Model

In [77]:
# Evaluate on test set
with torch.no_grad():
    model.eval()
    test_predictions = model(x_test_tensor).numpy()
    
# Calculate test RMSE (Root Mean Squared Error)
rmse = ((test_predictions - y_test.values) ** 2).mean() ** 0.5
print(f"Test RMSE: {rmse:.2f}")

# Calculate test MSE (Mean Square Error)
mse = mean_squared_error(y_test.values, test_predictions)
print(f"Test MSE: {mse}")

Test RMSE: 603.56
Test MSE: 223036.85574841473


In [79]:
# Test the model on some date
testing_date = "2023-08-15"
testing_data = data.loc[data["datetime"] == testing_date]

testing_parameters = scaler.transform(testing_data[features])

with torch.no_grad():
    input_features = torch.tensor([testing_parameters], dtype=torch.float64)
    predicted_count = model(input_features)

In [80]:
testing_data

,count,datetime,day_of_month,day_of_week,is_national_holiday,is_school_holiday,location,month,rain,temperature
591,1328.0,2023-08-15,15,1,True,True,lovrenska-jezera,8,0.0,16.254861


In [81]:
predicted_count

tensor([[[1595.2377]]], dtype=torch.float64)

In [83]:
# Generate for each date
predictions = []

for _, neki in data.iterrows():
    neki2 = data.loc[data["datetime"] == neki["datetime"]]
    parameters = scaler.transform(neki2[features])
    with torch.no_grad():
        input_features = torch.tensor([parameters], dtype=torch.float64)
        predicted_count = model(input_features)
        predictions.append(f"{neki['datetime']},{float(predicted_count)}\n")

with open(f"../predictions/{location}.csv", "w") as file:
    file.writelines(predictions)

### Save Model

In [86]:
# Save the scaler
dump(scaler, f"../models/{location}-{revision}.bin", compress=True)

# Save the model
torch.save(model.state_dict(), f"../models/{location}-{revision}.pth")